In [1]:
import string
from collections import OrderedDict
import numpy as np
from ordered_set import OrderedSet
import pymatrix

## Encryption

In [2]:
def key_text_rule(key):
    for j in range(len(key)):
        for i in range(len(key)):
            if ((i%2==0) and (i+1!=len(key))):
                if ((key[i]) == (key[i+1])): 
                    near = i+1
                    key = key[:near] + 'x' + key[near:]
                    break

    if (len(key)%2!=0):
        key = key[:len(key)+1] + 'z'
        return key
    else:
        return key

In [3]:
def text_to_matrix(dict1, text, n):
    list1 = []
    for i in text:
        list1.append(dict1[i])

    matrix = np.array(list1).reshape(n,n)
    return matrix

In [4]:
def encryption(small_dict, key, n):
    key_matrix = text_to_matrix(small_dict, key, n)
    key_matrix = np.matrix(key_matrix)
    
    main_encrypt_list = []

    for i in range(len(plain_text)):
        plain_list1 = []
        if ((i%2==0) and (i<=len(plain_text))):

            plain_list1.append(small_dict[plain_text[i]])
            plain_list1.append(small_dict[plain_text[i+1]])

            main_encrypt_list.append((np.dot(key_matrix, np.array(plain_list1).reshape(n,))) % 26)
            
    cipher_text = []
    dict_keys=list(small_dict.keys()) 

    for i in main_encrypt_list:

        val1 = i[0,0]
        val2 = i[0,1]

        cipher_text.append(dict_keys[val1])
        cipher_text.append(dict_keys[val2])

    cipher_text = ''.join(map(str,cipher_text))
    return (cipher_text, key_matrix)

## Decryption

In [5]:
def gcd(a, b):
    
    if(b == 0):
        return a
    else:
        return gcd(b, a % b)

In [6]:
def modulo_multiplicative_inverse(key_matrix_det):
    
    if (gcd(key_matrix_det,26)==1):
        
        if (key_matrix_det>27):
            key_matrix_det = key_matrix_det%26
            
        num = 1
        while((key_matrix_det * num) % 26 !=1):
            num+=1
        
        return num

    else:
        return 0 # GCD(det,26)!=1, then modulo multiplicative inverse --> Not found

In [7]:
def decryption(cipher_text, key_matrix):

    key_matrix_det = int(np.linalg.det(key_matrix))
    one_by_det = modulo_multiplicative_inverse(key_matrix_det)
    
    if one_by_det:    
        adj = (pymatrix.matrix(key_matrix.tolist())).adjoint()
        ## converting into numpy and int array

        key_matrix_adj = []
        for i in range(n):
            key_matrix_adj.append(adj[i])

        key_matrix_adj = np.array(key_matrix_adj).astype(int)
        key_matrix_adj

        for i in key_matrix_adj:
            if (i[0] < 0) : i[0] += 26
            if (i[1] < 0) : i[1] += 26

        key_inverse = key_matrix_adj * one_by_det

        main_decrypt_list = []
        for i in range(len(cipher_text)):

            plain_list1 = []
            if ((i%2==0) and (i<=len(cipher_text))):

                plain_list1.append(small_dict[cipher_text[i]])
                plain_list1.append(small_dict[cipher_text[i+1]])

                main_decrypt_list.append(np.round(np.dot(key_inverse, np.array(plain_list1).reshape(n,))) % 26)

        main_decrypt_list = np.int_(main_decrypt_list)

        original_text = []

        for i in main_decrypt_list:
            dict_keys=list(small_dict.keys()) 
            original_text.append(dict_keys[i[0]])
            original_text.append(dict_keys[i[1]])

        original_text = ''.join(map(str,original_text))

        return original_text
    
    else:
        return "GCD!=1, No Modulo Multiplicative Inverse"

In [8]:
if __name__ == '__main__':

    small_dict = dict()
    for index, letter in enumerate(string.ascii_lowercase):
           small_dict[letter] = index + 0
            
    plain_text = key_text_rule('welcome')
    print(plain_text)
    n = 4
    key = 'abcdefghijklmnop'
    
    cipher_text, key_matrix = encryption(small_dict, key, n)
    plain_text_dec = decryption(cipher_text,key_matrix)

welcomez


ValueError: cannot reshape array of size 2 into shape (4,)

In [ ]:
cipher_text

In [ ]:
plain_text_dec